# Convolutional Neural Network for Credit Card Applications (Model)

## Import Custom Library for Feed Forward Neural Network

In [1]:
import numpy as np

class ActivationDoesNotExist(Exception):
    """Valid activations are sigmoid, tanh, and relu, provided as a string"""
    pass

class InputDimensionNotCorrect(Exception):
    """Need to specify input dimension, i.e. input shape into the first layer"""
    pass

class LossFunctionNotDefined(Exception):
    """Loss function in cost() method not defined"""
    pass


class DenseLayer:
    """
    A class to define fully connected layers.
    """
    
    def __init__(self, inputDimension, units, activation='', randomMultiplier=0.01):
        """
        Constructor:
          inputDimension: number of input features
          units: number of neurons in the layer
          activation: activation function applied to layer
            - options: 'sigmoid', 'tanh', 'relu', ''
          randomMultiplier: multiplier applied to the random weights during initialization
        """
        self.weights, self.bias = self.initialize(inputDimension, units, randomMultiplier)
        if activation == 'sigmoid':
            self.activation = activation
            self.activationForward = self.sigmoid
            self.activationBackward = self.sigmoidGrad
        elif activation == 'relu':
            self.activation = activation
            self.activationForward = self.relu
            self.activationBackward = self.reluGrad
        elif activation == 'tanh':
            self.activation = activation
            self.activationForward = self.tanh
            self.activationBackward = self.tanhGrad
        elif activation != '':
            raise ActivationDoesNotExist
        else:
            self.activation = 'none'
            self.activationForward = self.linear
            self.activationBackward = self.linear
    
    def initialize(self, nx, nh, randomMultiplier):
        """
        Initializes weights randomly:
          nx: number of input features
          nh: number of units
          randomMultiplier: multiplier applied to the random weights during initialization
        returns:
          weights: the randomly initialized weights
          bias: the bias terms
        """
        weights = randomMultiplier * np.random.randn(nh, nx)
        bias = np.zeros([nh, 1])
        return weights, bias

    
    def sigmoid(self, Z):
        """
        Sigmoid activation function
        """
        A = 1 / (1 + np.exp(-Z))
        return A
        
    def sigmoidGrad(self, dA):
        """
        Differential of sigmoid function with chain rule applied
        """
        s = 1 / (1 + np.exp(-self.prevZ))
        dZ = dA * s * (1 - s)
        return dZ
    
    
    def relu(self, Z):
        """
        Relu activation function
        """
        A = np.maximum(0, Z)
        return A
        
    def reluGrad(self, dA):
        """
        Differential of relu function with chain rule applied
        """
        s = np.maximum(0, self.prevZ)
        dZ = (s>0) * 1 * dA
        return dZ 

        
    def tanh(self, Z):
        """
        Tanh activation function
        """
        A = np.tanh(Z)
        return A

    def tanhGrad(self, dA):
        """
        Differential of tanh function with chain rule applied
        """
        s = np.tanh(self.prevZ)
        dZ = (1 - s**2) * dA
        return dZ


    def linear(self, Z):
        """
        Placeholder when no activation function is used
        """
        return Z
        
    
    def forward(self, A):
        """
        Forward pass through layer
          A: input vector
        """
        Z = np.dot(self.weights, A) + self.bias
        self.prevZ = Z
        self.prevA = A
        A = self.activationForward(Z)
        return A
    
    
    def backward(self, dA):
        """
        Backward pass through layer
          dA: previous gradient
        """
        dZ = self.activationBackward(dA)
        m = self.prevA.shape[1]
        self.dW = 1 / m * np.dot(dZ, self.prevA.T)
        self.db = 1 / m * np.sum(dZ, axis=1, keepdims=True)
        prevdA = np.dot(self.weights.T, dZ)
        return prevdA
    
    
    def update(self, learning_rate):
        """
        Update weights using gradients from backward pass
          learning_rate: the learning rate used in the gradient descent
        """
        self.weights = self.weights - learning_rate * self.dW
        self.bias = self.bias - learning_rate * self.db

        
    def outputDimension(self):
        """
        Returns the output dimension for the next layer
        """
        return len(self.bias)

    
    def __repr__(self):
        """
        Used to print a pretty summary of the layer
        """
        act = 'none' if self.activation == '' else self.activation
        return f'Dense layer (nx={self.weights.shape[1]}, nh={self.weights.shape[0]}, activation={act})'



class NeuralNetwork:
    """
    Neural Network structure that holds our layers
    """
    
    def __init__(self, loss='cross-entropy', randomMultiplier = 0.01):
        """
        Constructor:
          loss: the loss function. Two are defined:
             - 'cross-entropy' and 'mean-square-error'
          randomMultiplier: multiplier applied to the random weights during initialization
        """
        self.layers=[]
        self.randomMultiplier = randomMultiplier
        if loss=='cross-entropy':
            self.lossFunction = self.crossEntropyLoss
            self.lossBackward = self.crossEntropyLossGrad
        elif loss=='mean-square-error':
            self.lossFunction = self.meanSquareError
            self.lossBackward = self.meanSquareErrorGrad
        else:
            raise LossFunctionNotDefined
        self.loss=loss


    def addLayer(self, inputDimension=None, units=1, activation=''):
        """
        Adds a Dense layer to the network:
          inputDimension: required when it is the first layer. otherwise takes dimensions of previous layer.
          units: number of neurons in the layer
          activation: activation function: valid choices are: 'sigmoid', 'tanh', 'relu', ''
        """
        if (inputDimension is None):
            if (len(self.layers)==0):
                raise InputDimensionNotCorrect
            inputDimension=self.layers[-1].outputDimension()
        layer = DenseLayer(inputDimension, units, activation, randomMultiplier= self.randomMultiplier)
        self.layers.append(layer)

    def crossEntropyLoss(self, Y, A, epsilon=1e-15):
        """
        Cross Entropy loss function
          Y: true labels
          A: final activation function (predicted labels)
          epsilon: small value to make minimize chance for log(0) error
        """
        m = Y.shape[1]
        loss = -1 * (Y * np.log(A + epsilon) + (1 - Y) * np.log(1 - A + epsilon))
        cost = 1 / m * np.sum(loss)
        return np.squeeze(cost)
            
    def crossEntropyLossGrad(self, Y, A):
        """
        Cross Entropy loss Gradient
          Y: true labels
          A: final activation function (predicted labels)
        """
        dA = -(np.divide(Y, A) - np.divide(1 - Y, 1 - A))
        return dA
    
    
    def meanSquareError(self, Y, A):
        """
        Mean square error loss function
          Y: true labels
          A: final activation function (predicted labels)
        """
        loss = np.square(Y - A)
        m = Y.shape[1]
        cost = 1 / m * np.sum(loss)
        return np.squeeze(cost)
    
    def meanSquareErrorGrad(self, Y, A):
        """
        Mean square error loss gradient
          Y: true labels
          A: final activation function (predicted labels)
        """
        dA = -2 * (Y - A)
        return dA

    
    def cost(self, Y, A):
        """
        Cost function wrapper
          Y: true labels
          A: final activation function (predicted labels)
        """
        return self.lossFunction(Y, A)

        
    def forward(self, X):
        """
        Forward pass through the whole model.
          X: input vector
        """
        x = np.copy(X)
        for layer in self.layers:
            x = layer.forward(x)
        return x
            
    
    def backward(self, A, Y):
        """
        backward pass through the whole model
          Y: true labels
          A: final activation function (predicted labels)
        """
        dA = self.lossBackward(Y, A)
        for layer in reversed(self.layers):
            dA = layer.backward(dA)
    
    
    def update(self, learning_rate=0.01):
        """
        Update weights and do a step of gradient descent for the whole model.
          learning_rate: learning_rate to use
        """
        for layer in self.layers:
            layer.update(learning_rate)
    
    
    def __repr__(self):
        """
        Pretty print the model
        """
        layrepr = ['  ' + str(ix+1)+' -> ' + str(x) for ix, x in enumerate(self.layers)]
        return '[\n' + '\n'.join(layrepr) + '\n]'
   
    
    def numberOfParameters(self):
        """
        Print number of trainable parameters in the model
        """
        n = 0
        for layer in self.layers:
            n += np.size(layer.weights) + len(layer.bias)
        print(f'There are {n} trainable parameters in the model.')

## Data Preprocessing for Neural Network

In [4]:
import pandas as pd

#Load the dataset to be used
df1 = pd.read_csv("updatedMergedApplicationCreditRecords.csv")
df1.head()

,Unnamed: 0,ID,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,YEARS_OLD,DAYS_EMPLOYED,MONTHS_BALANCE,STATUS
0,0,5008804,Y,Y,427500.0,Working,Higher education,Civil marriage,Rented apartment,32.0,4542.0,-0.0,1
1,1,5008804,Y,Y,427500.0,Working,Higher education,Civil marriage,Rented apartment,32.0,4542.0,1.0,1
2,2,5008804,Y,Y,427500.0,Working,Higher education,Civil marriage,Rented apartment,32.0,4542.0,2.0,1
3,3,5008804,Y,Y,427500.0,Working,Higher education,Civil marriage,Rented apartment,32.0,4542.0,3.0,1
4,4,5008804,Y,Y,427500.0,Working,Higher education,Civil marriage,Rented apartment,32.0,4542.0,4.0,1


In [10]:
#Begin by removing duplicates and pruning columns we determined in the Unsupervised Learning step to be unhelpful
df1 = df1.drop_duplicates("ID")
df1 = df1.drop(columns=["Unnamed: 0", "ID", "FLAG_OWN_CAR", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", "YEARS_OLD"])
df1.head()

,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,DAYS_EMPLOYED,MONTHS_BALANCE,STATUS
0,Y,427500.0,4542.0,-0.0,1
16,Y,427500.0,4542.0,-0.0,1
31,Y,112500.0,1134.0,-0.0,1
61,Y,270000.0,3051.0,-0.0,1
66,Y,270000.0,3051.0,22.0,1


In [18]:
#Make a OneHotEncoder for our FLAG_OWN_REALTY column

from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix

#Start OneHotEncoding Process
encoder = make_column_transformer((OneHotEncoder(), ["FLAG_OWN_REALTY"]), remainder="passthrough")
encoded = encoder.fit_transform(df1)
encoded_dataset = pd.DataFrame(encoded, columns=encoder.get_feature_names_out())

#Fix up the encoded column names
encoded_dataset.rename(columns= {"onehotencoder__FLAG_OWN_REALTY_N":"NO_REALTY",
                                "onehotencoder__FLAG_OWN_REALTY_Y":"YES_REALTY",
                                "remainder__AMT_INCOME_TOTAL":"TOTAL_INCOME",
                                "remainder__DAYS_EMPLOYED":"DAYS_EMPLOYED",
                                "remainder__MONTHS_BALANCE":"MONTHS_BALANCE",
                                "remainder__STATUS":"STATUS"}, inplace = True)

#encoded_dataset.head()

#Begin normalization process
minMax = MinMaxScaler()
encoded_dataset[["TOTAL_INCOME", "DAYS_EMPLOYED", "MONTHS_BALANCE"]] = minMax.fit_transform(encoded_dataset[["TOTAL_INCOME", "DAYS_EMPLOYED", "MONTHS_BALANCE"]])

#encoded_dataset.head()

,NO_REALTY,YES_REALTY,TOTAL_INCOME,DAYS_EMPLOYED,MONTHS_BALANCE,STATUS
0,0.0,1.0,0.258721,0.970676,0.000000,1.0
1,0.0,1.0,0.258721,0.970676,0.000000,1.0
2,0.0,1.0,0.055233,0.961730,0.000000,1.0
3,0.0,1.0,0.156977,0.966763,0.000000,1.0
4,0.0,1.0,0.156977,0.966763,0.366667,1.0



## Training and Testing the Neural Network


In [22]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

X = encoded_dataset.drop(["STATUS"], axis=1)
Y = encoded_dataset[["STATUS"]]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=5)

mlp = MLPClassifier(solver = "sgd", random_state = 42, activation = "logistic", learning_rate_init = 0.3, hidden_layer_sizes = (6, 3), max_iter = 500)
mlp.fit(X_train, Y_train)
predictions = mlp.predict(X_test)

/home/null/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [24]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix

#Confusion Matrix
print("Confusion Matrix")
confusion_matrix(Y_test, predictions)

#Accuracy and MSE of model
print("\n Accuracy of Model: ", accuracy_score(Y_test, predictions))
print("Mean Squared Error: ", mean_squared_error(Y_test, predictions))

#Precision and Recall
print("\n Precision and Recall")
print(classification_report(Y_test, predictions))

Confusion Matrix

 Accuracy of Model:  0.989687328579265
Mean Squared Error:  0.010312671420735052

 Precision and Recall
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        94
         1.0       0.99      1.00      0.99      9021

    accuracy                           0.99      9115
   macro avg       0.49      0.50      0.50      9115
weighted avg       0.98      0.99      0.98      9115



/home/null/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/null/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/null/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
import pickle
#Saving the model for future use
FFNN = "FFNN_model.sav"
pickle.dump(mlp, open(FFNN, "wb"))

#Load the model for use
#premade_model = pickle.load(open("FFNN_model.sav", "rb"))
#result = premade_model.score(X_test, Y_test)
#print(result)

0.989687328579265


## k-fold Cross Validation

## Hyperparameter Tuning